In [1]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

saved_path = '../saves/auc_maximization'

dls, labels = util.chexpert_data_loader(bs=32)

chexpert_learner_base = util.ChexpertLearner(dls, densenet121, n_out=len(labels),
                                        loss_func=BCEWithLogitsLossFlat(),
                                        model_dir=Path('auc_maximization'),
                                        metrics=[RocAucMulti(average=None),
                                                 RocAucMulti(average='weighted')])

In [4]:
# Save trained model
torch.save(chexpert_learner_base.learn.model.state_dict(), path/'chexpert_learner_base.pth')

# Create a new densenet121 and load the previous model's state_dict()
chexpert_learner_dam = densenet121().cuda()
chexpert_learner_dam.load_state_dict(torch.load(path/'chexpert_learner_base.pth'))

next(chexpert_learner_dam.parameters()).is_cuda

<function torchvision.models.densenet.densenet121(pretrained=False, progress=True, **kwargs)>

Copy `chextpert_learner_base` and replace it with a new head

In [48]:
from copy import deepcopy

# Perform a deepcopy of chexpert_learner_base model
chexpert_learner_dam = deepcopy(chexpert_learner_base.learn.model)

# Helper function to find the output shape
def get_output_shape(model, image_dim):
    return model(torch.rand(*(image_dim))).data.shape

# Compute number of input features for the new head 
image_dim = next(iter(dls.train))[0].shape
nf = get_output_shape(chexpert_learner_base.learn.model[0], next(iter(dls.train))[0].shape)[1]

# Replace with a new head
chexpert_learner_dam[1] = create_head(nf, len(labels))

In [49]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG

gamma = 500
weight_decay = 0
margin = 1.0

loss_func = AUCMLoss()
opt_func = PESG(chexpert_learner_dam, a=loss_func.a, b=loss_func.b, alpha=loss_func.alpha, gamma=gamma, margin=margin)

In [ ]:
def train(model, dls, loss_func, opt_func, max_epoch=2):
    for epoch in range(max_epoch):

        train_pred = []
        train_true = []
        model.train()    
        for i, (data, targets) in enumerate(dls.train):
            y_pred = model(data)
            loss = loss_func(y_pred, targets)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

            train_pred.append(y_pred.cpu().detach().numpy())
            train_true.append(targets.cpu().detach().numpy())
            
            if i == 2000:
                optimizer.lr = optimizer.lr/3
                optimizer.update_regularizer()

        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        train_auc = roc_auc_score(train_true, train_pred) 

        model.eval()
        test_pred = []
        test_true = [] 
        for j, (test_data, test_targets) in enumerate(dls.valid):
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_targets.numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        val_auc =  roc_auc_score(test_true, test_pred) 

        # print results
        print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}, lr:{:4f}".format(epoch, loss.item(), train_auc, val_auc, optimizer.lr ))